In [1]:
DEEPSEEK_API_KEY="your api_key"
BASE_URL="https://api.deepseek.com"

想象一下，现在你有这么一个功能，你创建了基于N个不同Prompt的机器人，你希望让模型自己选择使用哪个机器人回答你的问题，这种场景下路由就发挥作用了。  
Router Chain利用LLM基于用户question选择使用哪个机器人进行回复。  
MultiPromptChain:
* 1 Default Chain
* 2 LLMRouterChain
* 3 Destination Chains

In [2]:
from langchain_openai import ChatOpenAI
from langchain.chains.router import MultiPromptChain

In [3]:
physics_template = """You are a very smart physics professor. 
You are great at answering questions about physics in a concise and easy to understand manner. 
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. You are great at answering math questions. 
You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [11]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
]
llm = ChatOpenAI(model="deepseek-chat", api_key=DEEPSEEK_API_KEY, base_url=BASE_URL)

MultiPromptChain.from_prompts中存在详细源码，可以细细探查一番

In [18]:
chain = MultiPromptChain.from_prompts(llm, prompt_infos, verbose=True)

LLMChain选择physics chain回复

In [19]:
chain.invoke("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


{'input': 'What is black body radiation?',
 'text': 'Black body radiation refers to the electromagnetic radiation emitted by an idealized physical body called a "black body" that absorbs all incident electromagnetic radiation, regardless of frequency or angle of incidence. A black body is an idealized surface with no reflective properties; it absorbs and re-emits all the energy it receives.\n\nThe radiation emitted by a black body depends only on its temperature and not on the material\'s composition or other properties. This radiation is a continuous spectrum, meaning it covers a wide range of wavelengths, and its distribution follows a specific pattern known as Planck\'s law.\n\nAt low temperatures, most of the emitted radiation is in the infrared region of the electromagnetic spectrum, making it invisible to the human eye. As the temperature increases, the peak of the radiation shifts towards shorter wavelengths, eventually including visible light. This is why objects at higher temp

LLMChain选择math chain回复

In [20]:
chain.invoke("What is the first prime number greater than 40 such that one plus the prime number is divisible by 3")



> Entering new MultiPromptChain chain...
math: {'input': 'What is the first prime number greater than 40 such that one plus the prime number is divisible by 3'}
> Finished chain.


{'input': 'What is the first prime number greater than 40 such that one plus the prime number is divisible by 3',
 'text': 'To find the first prime number greater than 40 such that one plus the prime number is divisible by 3, we can follow these steps:\n\n1. **Identify the prime numbers greater than 40**: The prime numbers greater than 40 are 41, 43, 47, 53, 59, 61, 67, 71, etc.\n\n2. **Check the condition for each prime number**: We need to check if \\( p + 1 \\) is divisible by 3, where \\( p \\) is the prime number.\n\n   - For \\( p = 41 \\):\n     \\[\n     41 + 1 = 42\n     \\]\n     \\( 42 \\div 3 = 14 \\), which is an integer. So, 41 satisfies the condition.\n\n   - For \\( p = 43 \\):\n     \\[\n     43 + 1 = 44\n     \\]\n     \\( 44 \\div 3 = 14.6667 \\), which is not an integer. So, 43 does not satisfy the condition.\n\n   - For \\( p = 47 \\):\n     \\[\n     47 + 1 = 48\n     \\]\n     \\( 48 \\div 3 = 16 \\), which is an integer. So, 47 satisfies the condition.\n\n   - F

LLMChain选择default chain回复

In [21]:
chain.invoke("What is the name of the type of cloud that rains")



> Entering new MultiPromptChain chain...
None: {'input': 'What is the name of the type of cloud that produces rain?'}
> Finished chain.


{'input': 'What is the name of the type of cloud that produces rain?',
 'history': '',
 'text': 'Oh, that would be the cumulonimbus cloud! These clouds are quite fascinating, as they are associated with heavy rain, thunderstorms, and even hail. They form from the rising and cooling of moist air, which then condenses into towering cloud formations that can reach high into the atmosphere, sometimes even up to 50,000 feet! The name "cumulonimbus" comes from the Latin words "cumulus" meaning "heaped" and "nimbus" meaning "rain cloud." So, when you see those big, puffy, and often dark clouds looming in the sky, you can bet they\'re cumulonimbus clouds getting ready to bring some rain!'}